# Building a coffee machine energy system model 
## *Getting hands on building a simple energy system model* 

This example is designed for the beginners to understand MESSAGEix framework through an easy and simple example.

**Pre-requisites**
- Have succesfully installed *MESSAGEix*.


For information on how to install *MESSAGEix*, please refer to [Installation page](https://docs.messageix.org/en/stable/getting_started.html) and for getting *MESSAGEix* tutorials, please follow the steps mentioned in [Tutorials](https://docs.messageix.org/en/stable/tutorials.html).

Please refer to the [user guidelines](https://github.com/iiasa/message_ix/blob/master/NOTICE.rst)
for additional information on using *MESSAGEix*, including the recommended citation and how to name new models.

## *MESSAGEix: the mathematical paradigm*

At its core, *MESSAGEix* is an optimization problem:

> $\min \quad ~c^T \cdot x$  
> $~s.t. \quad A \cdot x \leq b$

More explicitly, the model...
- optimizes an **objective function**, nominally minimizing total **system costs**
- under a system of **constraints** (inequalities or equality conditions)

The mathematical implementation includes a number of features that make it particularly geared towards the modelling of *energy-water-land systems* in the context of *climate change mitigation and sustainable development*.

Throughout this document, the mathematical formulation follows the convention that
- decision **VARIABLES** ($x$) are capitalized
- input **parameters** ($A$, $b$) are lower case


## *A stylized reference energy system model for Westeros*

<img src="coffee_system.png">


# Let's start! 


## 1) Importing required packages
For this model message_ix and ixmp Python packages are required. For other models,maybe some more packages are required which can be scripted under this part. 

In [1]:
import ixmp                    
import message_ix 
import pandas as pd

The *MESSAGEix* model is built using the *ixmp* `Platform`. The `Platform` is your connection to a database for storing model input data and scenario results. For more information,refer to ix modelling platform documentation [here](https://docs.messageix.org/projects/ixmp/en/stable/index.html)

In [2]:
 mp = ixmp.Platform(name = 'local')

#### Building a new, empty MESSAGEix scenario

We assign our `model` snd `Scenario` a name and then store the  `Scenario` isntance to a variable. A `Scenario` instance will contain all the model input data and results.

In [3]:
model = 'cofee_energy_model' # model name
scenario = 'test' # scenario name 
scen = message_ix.Scenario(mp, model, scenario,  version= 'new')

Missing annotation for new Scenario cofee_energy_model/test


We can check the version of scenario instance using following command, if multiple scenario instances are stored in database

In [4]:
scen.version

0

We can also see what sets, 

## 2) Spatial & temporal detail


We start with assigining a node, in this case, we assume my_home and let's say we want to include 2020 & 2021 in our study.

In [5]:
scen.add_set('node', 'my_home')
scen.add_set('year', [2020, 2021])

## 3) Declaring members of the sets

Let's see a list of sets included in MESSAGEix framework. We will not use all sets in this example though. 

In [6]:
scen.set_list()

['year',
 'node',
 'technology',
 'relation',
 'emission',
 'land_scenario',
 'land_type',
 'lvl_spatial',
 'time',
 'lvl_temporal',
 'type_node',
 'type_tec',
 'type_year',
 'type_emission',
 'type_relation',
 'mode',
 'grade',
 'level',
 'commodity',
 'rating',
 'shares',
 'type_addon',
 'level_storage',
 'storage_tec',
 'map_spatial_hierarchy',
 'map_node',
 'map_temporal_hierarchy',
 'map_time',
 'cat_node',
 'cat_tec',
 'cat_year',
 'cat_emission',
 'type_tec_land',
 'cat_relation',
 'level_resource',
 'level_renewable',
 'level_stocks',
 'map_shares_commodity_total',
 'map_shares_commodity_share',
 'addon',
 'cat_addon',
 'map_tec_addon',
 'balance_equality',
 'map_tec_storage']

We start filling in the model's `commodities`, `levels`, `technologies`, and `modes` (i.e., modes of operation of technologies. In this case, we have only one technology,`coffee_maker`, and then information how coffee maker operates as shown in figure above.. This information defines how certain technologies operate. 

In [7]:
scen.add_set('technology', 'coffee_maker')
scen.add_set('commodity', ['water', 'electr', 'coffee_beans', 'coffee_cup']) # all members can be added at once
scen.add_set('level', ['pipe', 'storage', 'grid', 'useful'])
scen.add_set('mode', 'normal')    # modes of operation of a technology

Let's check if the sets are correctly added to `commodity`

In [8]:
scen.set('commodity')

0           water
1          electr
2    coffee_beans
3      coffee_cup
dtype: object

## 2. Adding data to parameters
Let's see a list of parameters in MESSAGEix framework

In [9]:
scen.par_list()

['commodity_stock',
 'demand',
 'resource_cost',
 'resource_remaining',
 'bound_extraction_up',
 'resource_volume',
 'technical_lifetime',
 'capacity_factor',
 'operation_factor',
 'min_utilization_factor',
 'inv_cost',
 'fix_cost',
 'var_cost',
 'output',
 'input',
 'abs_cost_new_capacity_soft_up',
 'abs_cost_new_capacity_soft_lo',
 'level_cost_new_capacity_soft_up',
 'level_cost_new_capacity_soft_lo',
 'abs_cost_activity_soft_up',
 'abs_cost_activity_soft_lo',
 'level_cost_activity_soft_up',
 'level_cost_activity_soft_lo',
 'bound_new_capacity_up',
 'bound_new_capacity_lo',
 'bound_total_capacity_up',
 'bound_total_capacity_lo',
 'bound_activity_up',
 'bound_activity_lo',
 'initial_new_capacity_up',
 'growth_new_capacity_up',
 'soft_new_capacity_up',
 'initial_new_capacity_lo',
 'growth_new_capacity_lo',
 'soft_new_capacity_lo',
 'initial_activity_up',
 'growth_activity_up',
 'soft_activity_up',
 'initial_activity_lo',
 'growth_activity_lo',
 'soft_activity_lo',
 'emission_factor',
 

We can also filter out parameters using basic Python fucntionality such as emission related parameters can be filtered out as;

In [10]:
[x for x in scen.par_list() if 'emission' in x]

['emission_factor',
 'historical_emission',
 'emission_scaling',
 'bound_emission',
 'tax_emission',
 'land_emission']

### 2.1) Adding data to parameter `demand`
We can always see the indexes of a parameter and make a table with those indexes (plus "value" and "unit"). The method is `idx_names()`


In [11]:
scen.idx_names("demand")

['node', 'commodity', 'level', 'year', 'time']

Let's add a Python dataframe based on these indexes

In [12]:
df = pd.DataFrame({'node': 'my_home',
                   'commodity': 'coffee_cup',
                   'level': 'useful',
                   'year': [2020, 2021],
                   'value': [450, 500],    # cup of coffee per year
                   'unit': '-',
                   'time': 'year'
                   })
df

,node,commodity,level,year,value,unit,time
0,my_home,coffee_cup,useful,2020,450,-,year
1,my_home,coffee_cup,useful,2021,500,-,year


Now, we add this dataframe to oru scenario instance

In [13]:
scen.add_par('demand', df)
scen.par('demand') # to check, if correctly added

,node,commodity,level,year,time,value,unit
0,my_home,coffee_cup,useful,2020,year,450.0,-
1,my_home,coffee_cup,useful,2021,year,500.0,-


Similar method for including an `output` dataframe

In [14]:
# Adding data to the parameter "output" for the technology
df = pd.DataFrame({'node_loc': 'my_home',
                   'node_dest': 'my_home',
                   'technology': 'coffee_maker',
                   'commodity': 'coffee_cup',
                   'level': 'useful',
                   'year_vtg': [2020, 2021],   # once installed can be active only one period
                   'year_act': [2020, 2021],
                   'mode': 'normal',
                   'value': 0.9,
                   'unit': '-',
                   'time': 'year',
                   'time_dest': 'year',
                    })
scen.add_par('output', df)

In [15]:
# Seeing the content of a parameter
scen.par('output')

,node_loc,technology,year_vtg,year_act,mode,node_dest,commodity,level,time,time_dest,value,unit
0,my_home,coffee_maker,2020,2020,normal,my_home,coffee_cup,useful,year,year,0.9,-
1,my_home,coffee_maker,2021,2021,normal,my_home,coffee_cup,useful,year,year,0.9,-


In [16]:
# Saving the data of this scenario
scen.commit('my first commit')   # This message is optional
            

If we want to add more stuff to this particular scenario instance, we can use the following command

In [17]:
# scen.check_out()

Time to solve our model!

In [18]:
scen.solve()

In [19]:
# Objective value (why is it zero?)
scen.var('OBJ')

{'lvl': 0.0, 'mrg': 0.0}

In [20]:
# Activity of coffee maker
scen.var('ACT')

,node_loc,technology,year_vtg,year_act,mode,time,lvl,mrg
0,my_home,coffee_maker,2020,2020,normal,year,500.000000,0.0
1,my_home,coffee_maker,2021,2021,normal,year,555.555556,0.0


Note: remember to close the platform before openning in another session

In [21]:
mp.close_db()

# Homework 
 
1. Make sure, the script we did in class runs correctly on your machine. Please let us know, in case of any issues. 

2. We have added only one `commodity` in the output.Try to add more atleast two more commodities in this model. 

*Hint: Start by adding `demand` of that `commodity` and then following same steps as we did for `coffee_cup`*.
